In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [ ]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

In [ ]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

In [4]:
def run_pca(mtx, mtx_path, suffix):
    mtx = hl.variant_qc(mtx)
    mtx = mtx.checkpoint(localfs_path+'variant_qced_'+suffix+mtx_path)
    mtx = hl.read_matrix_table(localfs_path+'variant_qced_'+suffix+mtx_path)
    for_pca = mtx.filter_rows(mtx.variant_qc.AF[1] > 0.05)
    pruned_variant_table = hl.ld_prune(for_pca.GT, r2=0.2, bp_window_size=500000)
    for_pca = for_pca.filter_rows(hl.is_defined(pruned_variant_table[for_pca.row_key]))

    for_pca = for_pca.checkpoint(localfs_path+'for_pca_20_'+suffix+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'for_pca_20_'+suffix+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(pruned_scores = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_20_'+suffix+mtx_path)
    
    return(mtx)

def run_pca_no_filter(mtx, mtx_path, suffix):
    
    for_pca = mtx.sample_rows(0.1)

    for_pca = for_pca.checkpoint(localfs_path+'subset_'+suffix+mtx_path)
    for_pca = hl.read_matrix_table(localfs_path+'subset_'+suffix+mtx_path)
    eigenvalues, pcs, _ = hl.hwe_normalized_pca(for_pca.GT, k=20)
    
    mtx = mtx.annotate_cols(scores_no_filter = pcs[mtx.s].scores)
    mtx = mtx.checkpoint(localfs_path+'after_pca_no_filters_'+suffix+mtx_path)
    
    return(mtx)

def remove_pca_outliers(mtx, field, last_score, mtx_path, suffix):
    
    mtx = mtx.annotate_globals(
            st = mtx.aggregate_cols(
                hl.agg.array_agg(
                    lambda pc: hl.agg.stats(pc),
                    mtx[field])
            )
        )

    mtx = mtx.annotate_cols(
            pc_outliers=hl.map(
                lambda s, st: hl.int((s > st['mean'] + (10 * st['stdev'])) | (s < st['mean'] - (10 * st['stdev']))),
                mtx[field][0:last_score],
                mtx.st
            )
        )

    mtx = mtx.filter_cols(
        hl.sum(mtx.pc_outliers) ==  0
    )
    
    mtx = mtx.checkpoint(localfs_path+'no_outliers'+mtx_path+suffix+'.mt')
    
    return(mtx)

### select matrices for skat:
1. sportsmen vs GTS 40
2. polish zeros vs GTS 40

In [9]:
mt_1 = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt')
mt_2 = hl.read_matrix_table(localfs_path+'poles_zerosunion.mt')

In [10]:
mts = [mt_1, mt_2]

In [12]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)

FatalError: NoClassDefFoundError: Could not initialize class java.util.zip.Adler32

Java stack trace:
java.lang.NoClassDefFoundError: Could not initialize class java.util.zip.Adler32
	at org.apache.spark.broadcast.TorrentBroadcast.calcChecksum(TorrentBroadcast.scala:112)
	at org.apache.spark.broadcast.TorrentBroadcast.$anonfun$writeBlocks$1(TorrentBroadcast.scala:146)
	at org.apache.spark.broadcast.TorrentBroadcast.$anonfun$writeBlocks$1$adapted(TorrentBroadcast.scala:144)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at org.apache.spark.broadcast.TorrentBroadcast.writeBlocks(TorrentBroadcast.scala:144)
	at org.apache.spark.broadcast.TorrentBroadcast.<init>(TorrentBroadcast.scala:95)
	at org.apache.spark.broadcast.TorrentBroadcastFactory.newBroadcast(TorrentBroadcastFactory.scala:34)
	at org.apache.spar

In [ ]:
for idx, mt in enumerate(mts):
    mt = mt.annotate_cols(
    group = hl.if_else(
            mt.s.contains('B'),
            'local_controls',
            hl.if_else(
                (mt.s.contains('NA') | mt.s.contains('HG')),
                '1kg_controls',
                hl.if_else(
                    mt.s.contains('polish'),
                    'polish_controls',
                    'GTS'
                )
            )
        )
    )
    
    mt = mt.annotate_cols(phenotypes = pheno[mt.s])
    mts[idx] = mt

In [ ]:
#sportsmen vs GTS 40
s_vs_gts = mts[0].filter_cols((mts[0].group == 'local_controls') | (mts[0].phenotypes.family == '.'))
s_vs_gts = s_vs_gts.filter_rows(hl.agg.any(s_vs_gts.GT.is_non_ref()))
s_vs_gts = s_vs_gts.checkpoint(localfs_path+'s_vs_gts.mt')    

# polish no zeros vs GTS 40
pw_vs_gts = mts[2].filter_cols((mts[1].group == 'polish_controls') | (mts[1].phenotypes.family == '.'))
pw_vs_gts = pw_vs_gts.filter_rows(hl.agg.any(pw_vs_gts.GT.is_non_ref()))
pw_vs_gts = pw_vs_gts.checkpoint(localfs_path+'pw_vs_gts.mt') 


In [ ]:
s_vs_gts = hl.read_matrix_table(localfs_path+'s_vs_gts.mt')  
pw_vs_gts = hl.read_matrix_table(localfs_path+'pw_vs_gts.mt') 

In [ ]:
mts = [
    s_vs_gts,
    pw_vs_gts
]
mts_2 = [
    s_vs_gts,
    pw_vs_gts
]
mts_paths = [
    's_vs_gts'
    'pw_vs_gts'
]

In [ ]:
for idx, mt in enumerate(mts):
    mts[idx] = run_pca(mt, mts_paths[idx])

In [ ]:
for mt_path in mts_paths:
    mt = hl.read_matrix_table(localfs_path+'after_pca_20_'+mt_path)
    mt.write(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/redone_after_pca_20_'+mt_path+'.mt',
        overwrite = True
    )

In [ ]:
mts_paths = [
    's_vs_gts'
    'pw_vs_gts'
]

mts = []

for mt_path in mts_paths:
    mt = hl.read_matrix_table(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/redone_after_pca_20_'+mt_path+'.mt'
    )
    
    mts.append(mt)

In [ ]:
mts_2 = []

for idx, mt in enumerate(mts):
     mts_2.append(run_pca_no_filter(mt, mts_paths[idx]))

In [ ]:
mts_paths = [
    's_vs_gts'
    'pw_vs_gts'
]

In [ ]:
mts = []

for mt_path in mts_paths:
    mt = hl.read_matrix_table(localfs_path+'after_pca_no_filters_'+mt_path)
    mts.append(mt)

In [ ]:
mts_2 = []

for mt_path in mts_paths:
    
    mt = hl.read_matrix_table(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/redone_after_pca_20_'+mt_path+'.mt'
    ) 
    
    
    mts_2.append(mt)

In [ ]:
for idx, mt in enumerate(mts):
    
    mt = mt.annotate_cols(scores_no_filter = mt.scores) #these are just the subsetted scores
    mt = mt.annotate_cols(pruned_scores = mts_2[idx].cols()[mt.s].scores) #these are the pruned scores
    mt = mt.naive_coalesce(1000)
    
    mt.write(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/'+mts_paths[idx]+'_pca.mt',
        overwrite = True
    ) 

In [5]:
mts_paths = [
    's_vs_gts'
    'pw_vs_gts'
]

mts = []

for path in mts_paths:

    mt = hl.read_matrix_table(
        '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/'+path+'_pca.mt'
    )
    
    mts.append(mt)

In [6]:
# remove PCA outliers

mtx_s_2 = []

for idx, mt in enumerate(mts):
    
    mtx = remove_pca_outliers(mt, 'pruned_scores', 2, mts_paths[idx], '_no_outs_pruned')
    
    mtx_2 = remove_pca_outliers(mt, 'scores_no_filter', 2, mts_paths[idx], '_no_outs_subs')
    
    mtx = mtx.drop(
        mtx['scores'],
        mtx['scores_no_filter'],
        mtx['pruned_scores']
    )
    
    mtx_2 = mtx_2.drop(
        mtx_2['scores'],
        mtx_2['scores_no_filter'],
        mtx_2['pruned_scores']
    )
    
    mts[idx] = mtx 
    mtx_s_2.append(mtx_2)

2023-05-04 09:53:20.629 Hail: INFO: wrote matrix table with 4753663 rows and 140 columns in 878 partitions to /localfs/2492284/no_outlierss_vs_gts_no_outs_pruned.mt
2023-05-04 09:55:28.051 Hail: INFO: wrote matrix table with 4753663 rows and 140 columns in 878 partitions to /localfs/2492284/no_outlierss_vs_gts_no_outs_subs.mt
2023-05-04 09:57:49.181 Hail: INFO: wrote matrix table with 3747953 rows and 1038 columns in 878 partitions to /localfs/2492284/no_outliersp0_vs_gts_no_outs_pruned.mt
2023-05-04 09:59:55.567 Hail: INFO: wrote matrix table with 3747953 rows and 1038 columns in 878 partitions to /localfs/2492284/no_outliersp0_vs_gts_no_outs_subs.mt
2023-05-04 10:02:07.366 Hail: INFO: wrote matrix table with 3031891 rows and 1038 columns in 878 partitions to /localfs/2492284/no_outlierspw_vs_gts_no_outs_pruned.mt
2023-05-04 10:04:06.156 Hail: INFO: wrote matrix table with 3031891 rows and 1038 columns in 878 partitions to /localfs/2492284/no_outlierspw_vs_gts_no_outs_subs.mt
2023-05-

In [7]:
for mt in mts:
    print(mt.count()) # option 1 - 2 last score, 10 sds

(4753663, 140)
(3747953, 1038)
(3031891, 1038)
(5174161, 540)


In [8]:
for mtx in mtx_s_2:
    print(mtx.count())

(4753663, 140)
(3747953, 1038)
(3031891, 1038)
(5174161, 540)


In [9]:
for mt in mts:
    print(mt.aggregate_cols(hl.agg.counter(mt.group))) # option 1 - 2 last score, 10 sds <- ok, let's keep this and see

{'GTS': 38, 'local_controls': 102}
{'GTS': 38, 'polish_controls': 1000}
{'GTS': 38, 'polish_controls': 1000}
{'1kg_controls': 501, 'GTS': 39}


In [10]:
for mtx in mtx_s_2:
    print(mtx.aggregate_cols(hl.agg.counter(mtx.group))) # option 1 - 2 last score, 10 sds <- ok, let's keep this and see

{'GTS': 38, 'local_controls': 102}
{'GTS': 38, 'polish_controls': 1000}
{'GTS': 38, 'polish_controls': 1000}
{'1kg_controls': 501, 'GTS': 39}


In [10]:
new_mt_paths = [
    'no_outlierseur_vs_gts_no_outs_pruned.mt'
    'no_outlierspw_vs_gts_no_outs_pruned.mt'
]

In [ ]:
for path in new_mt_paths:
    
    mt = hl.read_matrix_table(localfs_path+path)
    print(path)
    print(mt.count())

    mt = run_pca(mt, path, 'pca_round_two')
    print('run pca for '+path)
    mt = run_pca_no_filter(mt, path, 'pca_round_two')
    print('run pca for'+path+' no filters')

no_outliersp0_vs_gts_no_outs_pruned.mt
(3747953, 1038)


2023-05-05 12:44:27.264 Hail: INFO: wrote matrix table with 3747953 rows and 1038 columns in 878 partitions to /localfs/2492284/variant_qced_pca_round_twono_outliersp0_vs_gts_no_outs_pruned.mt
2023-05-05 12:44:30.512 Hail: INFO: ld_prune: running local pruning stage with max queue size of 216481 variants
2023-05-05 12:55:07.366 Hail: INFO: wrote table with 1646893 rows in 878 partitions to /localfs/2492284/tmp_hail/persist_tableVXoMh7d2Uu
2023-05-05 12:55:17.398 Hail: INFO: wrote table with 1646893 rows in 878 partitions to /localfs/2492284/tmp_hail/0UJo4NsnDKfhT84iYNSbTS


In [15]:
mts = []
for path in new_mt_paths:
    mt = hl.read_matrix_table(
        localfs_path+'after_pca_no_filters_'+'pca_round_two'+path)
    print(mt.count())
    mts.append(mt)

(5174161, 540)
(3747953, 1038)
(3031891, 1038)
(4753663, 140)
